In [2]:
import cv2
import numpy as np

In [2]:
for i in range (303 ,1100) :
    I = cv2.imread('pedestrian/input/in%06d.jpg' %i)
    cv2.imshow("I",I)
    cv2.waitKey (10)

cv2.destroyAllWindows()

In [3]:
step = 3

for i in range (303 ,1100, step) :
    I = cv2.imread('pedestrian/input/in%06d.jpg' %i)
    cv2.imshow("I",I)
    cv2.waitKey (10)

cv2.destroyAllWindows()

In [4]:
step = 1
prev_frame = cv2.imread('pedestrian/input/in%06d.jpg' % 300, cv2.IMREAD_GRAYSCALE) 

for i in range (303 ,1100, step) :
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i) 
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    #odejmowanie kolejnych ramek
    prev_frame_int = prev_frame.astype('int')
    curr_frame_int = IG.astype('int')
    diff = np.abs(curr_frame_int - prev_frame_int)

    #binaryzacja
    B = 1*(diff > 15)
    B = (B*255).astype('uint8')

    cv2.imshow("diff", B)
    cv2.waitKey (10)

cv2.destroyAllWindows()

In [5]:
step = 2
prev_frame = cv2.imread('pedestrian/input/in%06d.jpg' % 300, cv2.IMREAD_GRAYSCALE) 
kernel = np.ones((3, 3), np.uint8)

for i in range (303 ,1100, step) :
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    #odejmowanie kolejnych ramek
    prev_frame_int = prev_frame.astype('int')
    curr_frame_int = IG.astype('int')
    diff = np.abs(curr_frame_int - prev_frame_int)

    #binaryzacja
    B = 1*(diff > 15)
    B = (B*255).astype('uint8')

    #medianblur i morfologiczne
    B = cv2.medianBlur(B, 7)
    B = cv2.erode(B, kernel, iterations=2)
    B = cv2.dilate(B, kernel, iterations=2)

    cv2.imshow("diff", B)
    cv2.waitKey (10)

cv2.destroyAllWindows()

In [6]:
step = 2
prev_frame = cv2.imread('pedestrian/input/in%06d.jpg' % 300, cv2.IMREAD_GRAYSCALE) 
kernel = np.ones((3, 3), np.uint8) 

for i in range (303 ,1100, step) :
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i) 
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    #odejmowanie kolejnych ramek
    prev_frame_int = prev_frame.astype('int')
    curr_frame_int = IG.astype('int')
    diff = np.abs(curr_frame_int - prev_frame_int).astype('uint8')

    #binaryzacja
    B = 1*(diff > 15)
    B = (B*255).astype('uint8')

    #medianblur i morfologiczne
    B = cv2.medianBlur(B, 7)
    B = cv2.erode(B, kernel, iterations=1)
    B = cv2.dilate(B, kernel, iterations=2)

    #indeksacja
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(B)

    cv2.imshow("Labels", np.uint8(labels/retval * 255))
    cv2.waitKey (10)

cv2.destroyAllWindows()

In [47]:
step = 2
prev_frame = cv2.imread('pedestrian/input/in%06d.jpg' % 300, cv2.IMREAD_GRAYSCALE)
kernel = np.ones((3, 3), np.uint8)
min_area = 300

for i in range(303, 1100, step):
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    #odejmowanie kolejnych ramek
    prev_frame_int = prev_frame.astype('int')
    curr_frame_int = IG.astype('int')
    diff = np.abs(curr_frame_int - prev_frame_int)

    #binaryzacja
    B = 1 * (diff > 10)
    B = (B * 255).astype('uint8')

    #medianblur i morfologiczne
    B = cv2.medianBlur(B, 7)
    B = cv2.erode(B, kernel, iterations=1)
    B = cv2.dilate(B, kernel, iterations=1)

    #indeksacja
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(B)

    I_VIS = I.copy()

    if stats.shape[0] > 1:  #czy sa obiekty
        tab = stats[1:, 4]  #pola obiektow bez tla
        pi = np.argmax(tab) + 1  #indeks najwiekszego obiektu

        if stats[pi, 4] > min_area:  #optymalizacja: sprawdzamy czy obiekt jest wystarczajaco duzy
            #rysowanie prostokata na najwiekszym obiekcie
            cv2.rectangle(I_VIS, (stats[pi, 0], stats[pi, 1]),
                        (stats[pi, 0] + stats[pi, 2], stats[pi, 1] + stats[pi, 3]),
                        (255, 0, 0), 2)

            #pole obiektu
            cv2.putText(I_VIS, "%f" % stats[pi, 4], (stats[pi, 0], stats[pi, 1]),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0))

            #indeks obiektu
            cv2.putText(I_VIS, "%d" % pi, (int(centroids[pi, 0]), int(centroids[pi, 1])),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))

    cv2.imshow("Detected Object", I_VIS)
    cv2.waitKey(10)

    prev_frame = IG

cv2.destroyAllWindows()


In [48]:
with open('pedestrian/temporalROI.txt', 'r') as f:
    roi_start, roi_end = map(int, f.readline().split())

#globalne liczniki
TP, TN, FP, FN = 0, 0, 0, 0

step = 2
prev_frame = cv2.imread('pedestrian/input/in%06d.jpg' % 300, cv2.IMREAD_GRAYSCALE)
kernel = np.ones((3, 3))

for i in range(303, 1100, step):
    if i < roi_start or i > roi_end:  #czy klatka w zakresie roi
        continue

    I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    #maska referencyjna
    GT = cv2.imread('pedestrian/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)

    if GT is None:
        continue

    #odejmowanie kolejnych ramek
    prev_frame_int = prev_frame.astype('int')
    curr_frame_int = IG.astype('int')
    diff = np.abs(curr_frame_int - prev_frame_int)

    #binaryzacja
    B = (diff > 10).astype('uint8') * 255

    #medianblur i morfologiczne
    B = cv2.medianBlur(B, 7)
    B = cv2.erode(B, kernel, iterations=1)
    B = cv2.dilate(B, kernel, iterations=1)

    #konwersja maski na wartosci binarne
    GTB = (GT > 128).astype('uint8') * 255

    TP += np.sum(np.logical_and(B == 255, GTB == 255))
    TN += np.sum(np.logical_and(B == 0, GTB == 0))
    FP += np.sum(np.logical_and(B == 255, GTB == 0))
    FN += np.sum(np.logical_and(B == 0, GTB == 255))

    cv2.imshow("Detected Objects", B)
    cv2.imshow("Ground Truth", GTB)
    cv2.waitKey(10)

    prev_frame = IG

cv2.destroyAllWindows()

# Obliczenie metryk P, R i F1
P = TP/(TP+FP) if (TP+FP) > 0 else 0
R = TP/(TP+FN) if (TP+FN) > 0 else 0
F1 = 2*P*R/(P+R) if (P+R) > 0 else 0

print(f"Precision (P): {P:.4f}")
print(f"Recall (R): {R:.4f}")
print(f"F1-score: {F1:.4f}")


Precision (P): 0.7660
Recall (R): 0.7257
F1-score: 0.7453


In [43]:
with open('highway/temporalROI.txt', 'r') as f:
    roi_start, roi_end = map(int, f.readline().split())

#globalne liczniki
TP, TN, FP, FN = 0, 0, 0, 0

step = 2
prev_frame = cv2.imread('highway/input/in%06d.jpg' % 469, cv2.IMREAD_GRAYSCALE)
kernel = np.ones((3, 3))

for i in range(470, 1700, step):
    if i < roi_start or i > roi_end:  #czy klatka w zakresie roi
        continue

    I = cv2.imread('highway/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    #maska referencyjna
    GT = cv2.imread('highway/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)

    if GT is None:
        continue

    #odejmowanie kolejnych ramek
    prev_frame_int = prev_frame.astype('int')
    curr_frame_int = IG.astype('int')
    diff = np.abs(curr_frame_int - prev_frame_int)

    #binaryzacja
    B = (diff > 10).astype('uint8') * 255

    #medianblur i morfologiczne
    B = cv2.medianBlur(B, 7)
    B = cv2.erode(B, kernel, iterations=1)
    B = cv2.dilate(B, kernel, iterations=1)

    #konwersja maski na wartosci binarne
    GTB = (GT > 128).astype('uint8') * 255

    TP += np.sum(np.logical_and(B == 255, GTB == 255))
    TN += np.sum(np.logical_and(B == 0, GTB == 0))
    FP += np.sum(np.logical_and(B == 255, GTB == 0))
    FN += np.sum(np.logical_and(B == 0, GTB == 255))

    # cv2.imshow("Detected Objects", B)
    # cv2.imshow("Ground Truth", GTB)
    # cv2.waitKey(10)

    prev_frame = IG  

cv2.destroyAllWindows()

P = TP/(TP+FP) if (TP+FP) > 0 else 0
R = TP/(TP+FN) if (TP+FN) > 0 else 0
F1 = 2*P*R/(P+R) if (P+R) > 0 else 0

print(f"Precision (P): {P:.4f}")
print(f"Recall (R): {R:.4f}")
print(f"F1-score: {F1:.4f}")


Precision (P): 0.8659
Recall (R): 0.7997
F1-score: 0.8315


In [46]:
with open('office/temporalROI.txt', 'r') as f:
    roi_start, roi_end = map(int, f.readline().split())

#globalne liczniki
TP, TN, FP, FN = 0, 0, 0, 0

step = 2
prev_frame = cv2.imread('office/input/in%06d.jpg' % 569, cv2.IMREAD_GRAYSCALE)
kernel = np.ones((5, 5)) 

for i in range(570, 1100, step):
    if i < roi_start or i > roi_end:  #czy klatka w zakresie roi
        continue

    I = cv2.imread('office/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    #maska referencyjna
    GT = cv2.imread('office/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)

    if GT is None:
        continue 

    #odejmowanie kolejnych ramek
    prev_frame_int = prev_frame.astype('int')
    curr_frame_int = IG.astype('int')
    diff = np.abs(curr_frame_int - prev_frame_int)

    #binaryzacja
    B = (diff > 5).astype('uint8') * 255

    #medianblur i morfologiczne
    B = cv2.medianBlur(B, 7)
    B = cv2.erode(B, kernel, iterations=1)
    B = cv2.dilate(B, kernel, iterations=3)

    #konwersja maski na wartosci binarne
    GTB = (GT > 128).astype('uint8') * 255

    TP += np.sum(np.logical_and(B == 255, GTB == 255))
    TN += np.sum(np.logical_and(B == 0, GTB == 0))
    FP += np.sum(np.logical_and(B == 255, GTB == 0))
    FN += np.sum(np.logical_and(B == 0, GTB == 255))

    # cv2.imshow("Detected Objects", B)
    # cv2.imshow("Ground Truth", GTB)
    # cv2.waitKey(10)

    prev_frame = IG

cv2.destroyAllWindows()

P = TP/(TP+FP) if (TP+FP) > 0 else 0
R = TP/(TP+FN) if (TP+FN) > 0 else 0
F1 = 2*P*R/(P+R) if (P+R) > 0 else 0

print(f"Precision (P): {P:.4f}")
print(f"Recall (R): {R:.4f}")
print(f"F1-score: {F1:.4f}")


Precision (P): 0.5288
Recall (R): 0.5417
F1-score: 0.5352
